<a href="https://colab.research.google.com/github/vibhuverma17/AUTOML/blob/main/H2O_Setup_and_Base_Run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# THIS COMMANF MIGHT BE RELEVANT ONLY ON GPU NODES, THIS NOTEBOOK RUNS ON CPU AS WELL
# IF RUNNING ON CPU PLEASE COMMENT THIS COMMAND WHENEVER YOU SEE IT

!nvidia-smi

In [ ]:
!pip install h2o
!apt-get install default-jre
!java -version

In [ ]:
### THIS STARTS AN H2O CLUSTER
import h2o
from h2o.automl import H2OAutoML
h2o.init()

## AUTOML GIVES US A DEPLOYMENT READY CODE
- MOJO
- POJO
- BINARY FORMAT



- USES GPU FOR XGBOOST

In [ ]:
churn_df = h2o.import_file('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
churn_df.describe()

In [ ]:
churn_train,churn_test,churn_valid = churn_df.split_frame(ratios=[.7, .15])

y = "Churn"
x = churn_df.columns
x.remove(y)
x.remove("customerID")

In [ ]:
aml = H2OAutoML(max_models = 10, seed = 10, exclude_algos = ["StackedEnsemble", "DeepLearning"], verbosity="info", nfolds=0)
aml.train(x = x, y = y, training_frame = churn_train, validation_frame=churn_valid)

In [ ]:
!nvidia-smi

In [ ]:
lb = aml.leaderboard

In [ ]:
lb

In [ ]:
churn_pred=aml.leader.predict(churn_test)

In [ ]:
### THESE ARE REAL PREDICTIONS BASED ON THE BEST MODEL
churn_pred.head()

In [ ]:
aml.leader.model_performance(churn_test)

In [ ]:
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
#se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])
#metalearner = h2o.get_model(se.metalearner()['name'])

In [ ]:
model_ids

In [ ]:
h2o.get_model([mid for mid in model_ids if "XGBoost" in mid][0])

In [ ]:
out = h2o.get_model([mid for mid in model_ids if "XGBoost" in mid][0])

In [ ]:
out.model_performance(churn_test)

In [ ]:
out.params

In [ ]:
out.convert_H2OXGBoostParams_2_XGBoostParams()

In [ ]:
out

In [ ]:
out.varimp_plot()

In [ ]:
aml.leader.download_mojo(path = "./")